In [ ]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.types import StringType
from snowflake.snowpark.types import IntegerType
from config import connection_parameters


In [ ]:
#Definir sesión y objetos Snowflake
sesion = Session.builder.configs(connection_parameters).create()
if sesion != None:
    print("Conectado")
    sesion.use_database('inegi')
    print(sesion.sql("select current_warehouse(), current_database(), current_role()").collect()) 
else:
    print("Error de conexión")

In [ ]:
#Crear vista 
query = "create or replace  view INEGI_DATA as select " + \
 "v:ENTIDAD::int as entidad," + \
 "v:MUN::int as municipio," + \
 "v:NOM_MUN::string as nom_municipio," + \
 "v:LOC::string as localidad," + \
 "v:NOM_LOC::string as nom_localidad," + \
 "v:LONGITUD::float as longitud," + \
 "v:LATITUD::float as latitud," + \
 "v:ALTITUD::int as altitud," + \
 "v:POBTOT::int as pob_total," + \
 "v:POBFEM::int as pob_fem," + \
 "v:POBMAS::int as pob_masc," + \
 "v:PCON_DISC::int as pob_discapacidad," + \
 "v:GRAPROES::int as pob_escolaridad," + \
 "v:GRAPROES_F::int as pob_esco_fem," + \
 "v:GRAPROES_M::int as pob_esco_masc," + \
 "v:PSINDER::int as pob_sssalud," + \
 "v:PDER_SS::int as pob_cssalud," + \
 "v:VIVTOT::int as total_vivienda," + \
 "v:TVIVHAB::int total_habitada," + \
 "v:VPH_INTER::int as hab_internet " + \
 "from INEGI_RAW;"
sesion.sql(query).collect()



In [ ]:
#UDF declaración
entidad_udf = sesion.udf.register_from_file(file_path='@inegi/entidad.py',func_name='nom_entity',return_type=StringType(),input_types=[IntegerType()],is_permanent=True, name="nom_entity",stage_location="@inegi")

In [13]:
#Vista con totales por entidad aplicando UDF para convertir número de entidad x nombre entidad
viewquery = "CREATE OR REPLACE VIEW INEGI_MAPA AS " + \
" WITH poblacion_lat AS ( SELECT latitud AS latitude, longitud AS longitude, nom_entity(entidad) as nom_entity," + \
" ROW_NUMBER() OVER(PARTITION BY nom_entity ORDER BY nom_entity DESC) AS row_number FROM INEGI_DATA)," + \
" poblacion_t AS ( SELECT sum(pob_total) AS poblacion_total,nom_entity(entidad) AS nom_entity FROM INEGI_DATA " + \
" GROUP BY entidad ORDER BY poblacion_total DESC )" + \
" SELECT pl.nom_entity,pt.poblacion_total AS total_population, pl.latitude, pl.longitude" + \
" FROM poblacion_lat pl LEFT JOIN poblacion_t pt ON pl.nom_entity = pt.nom_entity" + \
" WHERE row_number = 1;"
sesion.sql(viewquery).collect()

[Row(status='View INEGI_MAPA successfully created.')]

In [14]:
#Validar la vista solo con totales por entidad
df_entidad = sesion.table("INEGI_MAPA")
df_entidad.show()

----------------------------------------------------------------------------------------
|"NOM_ENTITY"          |"TOTAL_POPULATION"  |"LATITUDE"          |"LONGITUDE"          |
----------------------------------------------------------------------------------------
|BAJA CALIFORNIA SUR   |798447              |23.466304166666667  |-109.72155111111111  |
|DURANGO               |1832650             |23.07258638888889   |-104.96738694444444  |
|OAXACA                |4132148             |17.465217777777777  |-95.23456138888889   |
|PUEBLA                |6583278             |20.03848888888889   |-97.27450888888889   |
|SAN LUIS POTOSI       |2822255             |22.18987111111111   |-98.4165488888889    |
|TLAXCALA              |1342977             |19.404180277777776  |-98.04882305555556   |
|COAHUILA DE ZARAGOZA  |3146771             |25.794352777777778  |-103.08951666666667  |
|BAJA CALIFORNIA       |3769020             |30.096982777777775  |-115.6571538888889   |
|SONORA              

In [ ]:
sesion.close()